<a href="https://colab.research.google.com/github/Sripriya10/aero-mech-projects/blob/main/QR_Code_Detection_Python_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import time
from picamera2 import Picamera2
from pymavlink import mavutil # Corrected import
import navutil # Corrected import

qcd = cv2.QRCodeDetector()
cam = Picamera2()

# Configure camera preview
# Set main camera output resolution to 640x480 pixels
cam.preview_configuration.main.size = (640, 480)
# Set the pixel format to RGB888 for full color
cam.preview_configuration.main.format = "RGB888"
cam.configure("preview") # Apply the preview configuration
cam.start() # Start the camera preview

# Establish MAVLink connection to the flight controller
# Replace '/dev/ttyACM0' with the correct serial port for your setup (e.g., '/dev/ttyUSB0' on some systems)
# Baud rate is set to 57600, a common value for telemetry modules
c = mavutil.mavlink_connection('/dev/ttyACM0', baud=57600)
c.wait_heartbeat() # Wait for a heartbeat signal from the flight controller to confirm connection

time.sleep(1) # Give the camera a moment to initialize and settle

# Set camera controls for autofocus and lens position
# "AfMode": 2 typically enables continuous autofocus
# "LensPosition": 10.0 sets the lens to a specific focal distance (adjust as needed for your setup)
cam.set_controls({"AfMode": 2, "LensPosition": 10.0})

# Define video writer for saving the output video
# 'output.mp4' is the name of the output video file
# cv2.VideoWriter_fourcc(*'mp4v') specifies the MP4V codec for MP4 files
# 20.0 is the frames per second (FPS) for the output video
# (640, 480) is the resolution of the output video, matching the camera's capture resolution
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, 20.0, (640, 480))

# Function to send RC (Radio Control) override commands to the drone
# This function is used to control servo motors or other channels on the drone
# 'val' is the pulse width modulation (PWM) value for the specified channel
# Channels 1-7 are set to 0 (neutral/no override)
# 'val' is applied to channel 9 (index 8), which is often used for auxiliary functions like payload release servos
def switch_rc(val):
    c.mav.rc_channels_override_send(
        c.target_system,
        c.target_component,
        0, 0, 0, 0, 0, 0, 0, # Channels 1-7 (roll, pitch, yaw, throttle, aux1, aux2, aux3)
        val, # Channel 9 (aux4) - typically used for servo control
        0, 0, 0, 0, 0, 0 # Remaining channels (10-16)
    )

# Function to send MAVLink status text messages to the ground control station (GCS)
# 'txt' is the string message to be sent
# mavutil.mavlink.MAV_SEVERITY_NOTICE sets the severity level of the message
def send_message(txt):
    c.mav.statustext_send(mavutil.mavlink.MAV_SEVERITY_NOTICE, txt.encode("utf-8"))

scanned = 0 # Flag to ensure the servo action (payload drop) happens only once per QR code scan

send_message("rpi Running") # Send an initial status message to the GCS

# Main loop for video capture and QR code detection
while True:
    img = cam.capture_array() # Capture a single frame from the camera as a NumPy array

    # Detect and decode QR codes in the captured image
    # 'data' will contain the decoded string from the QR code
    # 'box' will contain the coordinates of the QR code's bounding box
    data, box, _ = qcd.detectAndDecode(img)

    # Check if any QR code data was successfully detected
    if len(data) != 0:
        print(data) # Print the decoded QR code data to the console

        # If a QR code is detected for the first time (scanned flag is 0)
        if scanned == 0:
            # Loop to control the servo for payload release
            # This loop sends RC override commands to move the servo from 2000 to 1600 PWM value
            # This range suggests a movement from a closed to an open position for a two-door mechanism
            for i in range(2000, 1600, -1):
                switch_rc(i)
                # It's highly recommended to add a small delay here (e.g., time.sleep(0.01))
                # to allow the servo to move smoothly and for MAVLink commands to be processed.
                # The original code had problematic 'i=199' and 'i-=100' lines which have been removed.
            send_message(data) # Send the decoded QR data as a MAVLink message to the GCS
            scanned = 1 # Set the flag to 1 to prevent re-triggering the servo action for the same QR code

        # If a QR code's bounding box is detected, draw a rectangle around it
        if box is not None and len(box) > 0:
            # Get the coordinates of the first detected QR code box
            box = box[0].astype(int)
            # Find the minimum and maximum x, y coordinates of the bounding box
            x_min, y_min = box.min(axis=0)
            x_max, y_max = box.max(axis=0)
            # Draw a green rectangle around the detected QR code
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)

            # Put the decoded QR code data as text on the image
            # Positioned at (100, 100), using DUPLEX font, green color, and thickness 2
            cv2.putText(img, data, (100, 100), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow("img", img) # Display the processed image in a window named "img"
    out.write(img) # Write the current frame to the output video file

    # Check for user input to exit the loop
    # If 'q' key is pressed (0xFF is a mask for the key code), break the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release all resources when the loop exits
cam.stop() # Stop the camera preview
cv2.destroyAllWindows() # Close all OpenCV windows